# Shannon's Theorem and Probabilistic Noise

So far, we have looked at codes from the **Hamming perspective**, where we assume a "worst-case" scenario: up to a certain number of errors (*t*) will occur, and our code must be able to handle it. This is an adversarial model.

This notebook introduces a different perspective. Instead of a worst-case scenario, Shannon modeled noise **stochastically**, or **probabilistically**. He viewed communication as a random process where errors happen with a certain likelihood, not with a guarantee.

## The Communication Process

Shannon's framework separates the problem of communication into two distinct parts, as shown in the general model below.

<img src="./imgs/image_channel_diagram.png" alt="Generic model of a communication system" width="600"/>

1. **Source Coding (Compression):** This part deals with the message itself before noise is a factor. The goal of the **Source Encoder** is to remove redundancy from the original data to make it as compact as possible. The theoretical limit of this compression is related to the **entropy** of the source. Think of this as zipping a file.

2. **Channel Coding (Error Correction):** This is the part we've been focused on. After the message is compressed, the **Channel Encoder** adds "smart" redundancy back in. This redundancy is not a simple repetition; it's structured (like in an RS code) to protect the data from errors that occur when it's sent over the noisy **Channel**.

Shannon showed that these two problems can be studied separately. For the rest of this chapter, we will ignore source coding and focus exclusively on **channel coding** in a world where the noise is random.

## Shannon's Noise Models: From BSC to Channel Capacity

Shannon's framework begins with a formal model of a noisy channel. It consists of three key parts:

1. **Input alphabet** $\,\mathcal{X}\,$ — the set of symbols we can send.
2. **Output alphabet** $\,\mathcal{Y}\,$ — the set of symbols we might receive.
3. **Transition matrix** $\,M\,$ — contains crossover probabilities. The entry $M(x,y)$ is the probability of receiving symbol $y$ given that we sent symbol $x$, written as $\Pr(y \mid x)$.

Channels are typically assumed to be **memoryless**, meaning the noise on each transmitted symbol is an independent event.

### Common Stochastic Channels

Let's look at a specific channel model.

#### Binary Symmetric Channel (BSC)

This is the most fundamental model. The input and output alphabets are both binary:
$\,\mathcal{X} = \mathcal{Y} = \{0,1\}.$

- A bit is transmitted correctly (0 stays 0, 1 stays 1) with probability $1 - p$.
- A bit is flipped (0 becomes 1, 1 becomes 0) with probability $p$. This $p$ is the **crossover probability**.

The channel can be visualized as:

<img src="./imgs/image_bsc.png" alt="Binary Symmetric Channel Diagram" width="300"/>

The corresponding transition matrix is:

$$
M = \begin{pmatrix}
1 - p & p \\
p & 1 - p
\end{pmatrix}
$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def simulate_bsc(codeword, p):
    """Simulates passing a codeword through a Binary Symmetric Channel."""
    received_word = []
    for bit in codeword:
        if np.random.rand() < p:
            # The bit is flipped
            received_word.append(1 - bit)
        else:
            # The bit is transmitted correctly
            received_word.append(bit)
    return np.array(received_word)

# Example
original_codeword = np.array([0, 1, 0, 1, 1, 1, 0])
crossover_prob = 0.1
received_codeword = simulate_bsc(original_codeword, crossover_prob)

print(f"Original codeword: {original_codeword}")
print(f"Received codeword: {received_codeword} (after BSC with p={crossover_prob})")

Original codeword: [0 1 0 1 1 1 0]
Received codeword: [0 1 1 1 1 0 0] (after BSC with p=0.1)


#### q-ary Symmetric Channel (qSC)

This model generalizes the BSC to an alphabet of any size $q \ge 2$.

- A symbol is transmitted correctly with probability $1 - p$.
- If an error occurs, the original symbol is transformed into any of the other $q - 1$ incorrect symbols with equal probability $\,\frac{p}{q-1}$.

---

#### Binary Erasure Channel (BEC)

In the BEC, bits are never flipped, but they can be completely lost or “erased”.

- The input alphabet is $\mathcal{X} = \{0,1\}$.
- The output alphabet is $\mathcal{Y} = \{0,1,?\}$, where “?” denotes an **erasure**.
- A bit is transmitted correctly with probability $1 - a$.
- A bit is erased with probability $a$.

<img src="./imgs/image_bec.png" alt="Binary Erasure Channel Diagram" width="200"/>

---

#### Binary Input Additive Gaussian White Noise (BIAGWN)

This is a crucial model for **continuous** channels, like radio waves.

- The input is binary, typically $\mathcal{X}=\{-1,1\}$.
- The output $\mathcal{Y}$ is the set of all real numbers ($\mathbb{R}$).
- The channel adds random noise drawn from a Gaussian (Normal) distribution to the input.  
  The probability density of receiving $y$ given $x$ is
  $$
  \Pr(y \mid x) \;=\; \frac{1}{\sigma\sqrt{2\pi}} \exp\!\left(-\frac{(y-x)^2}{2\sigma^2}\right).
  $$